In [1]:
!pip3 install python-chess

In [2]:
!pip install patool

In [3]:
import sys
print(sys.executable)

/Users/nguyennhatphong/Documents/ChessTrainAI/myenv/bin/python3.12


In [4]:
!pip install numpy


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import chess
import chess.engine
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import patoolib
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def generate_bitmaps(fen):
    board = chess.Board(fen)
    
    piece_index_map = {
        'k': 0,  'K': 6,
        'q': 1,  'Q': 7,
        'r': 2,  'R': 8,
        'n': 3,  'N': 9,
        'b': 4,  'B': 10,
        'p': 5,  'P': 11
    }

    material_value_map = {
        'k': 0,  'K': 0,
        'q': -9, 'Q': 9,
        'r': -5, 'R': 5,
        'n': -3, 'N': 3,
        'b': -3, 'B': 3,
        'p': -1, 'P': 1
    }

    file_mapping = 'abcdefgh'
    bitmap = np.zeros((14, 8, 8))

    fen_parts = fen.split(' ')
    position_data = fen_parts[0]

    row, col = 0, 0
    white_material, black_material = 0, 0

    for char in position_data:
        if char.isdigit():
            col += int(char)
        elif char == '/':
            row += 1
            col = 0
        else:
            piece_index = piece_index_map[char]
            value = 1 if char.isupper() else -1
            
            bitmap[piece_index, row, col] = value

            if char.isupper():
                white_material += material_value_map[char]
            else:
                black_material += material_value_map[char]

            col += 1

    legal_moves_layer = 12 if board.turn else 13
    opponent_moves_layer = 13 if board.turn else 12

    for move in board.legal_moves:
        target_col = file_mapping.index(str(move)[2])
        target_row = int(str(move)[3]) - 1
        bitmap[legal_moves_layer, target_row, target_col] = 1

    board.turn = not board.turn

    for move in board.legal_moves:
        target_col = file_mapping.index(str(move)[2])
        target_row = int(str(move)[3]) - 1
        bitmap[opponent_moves_layer, target_row, target_col] = 1

    bitmap[12] = np.flipud(bitmap[12])
    bitmap[13] = np.flipud(bitmap[13])

    return bitmap, white_material, black_material

def generate_compressed_bitmaps(fen):
    board = chess.Board(fen)

    piece_index_map = {
        'k': 0,  'K': 0,
        'q': 1,  'Q': 1,
        'r': 2,  'R': 2,
        'n': 3,  'N': 3,
        'b': 4,  'B': 4,
        'p': 5,  'P': 5
    }

    material_value_map = {
        'k': 0,  'K': 0,
        'q': -9, 'Q': 9,
        'r': -5, 'R': 5,
        'n': -3, 'N': 3,
        'b': -3, 'B': 3,
        'p': -1, 'P': 1
    }

    file_mapping = 'abcdefgh'
    bitmap = np.zeros((8, 8, 8))

    fen_parts = fen.split(' ')
    position_data = fen_parts[0]

    row, col = 0, 0
    white_material, black_material = 0, 0

    for char in position_data:
        if char.isdigit():
            col += int(char)
        elif char == '/':
            row += 1
            col = 0
        else:
            piece_index = piece_index_map[char]
            value = 1 if char.isupper() else -1
            
            bitmap[piece_index, row, col] = value

            if char.isupper():
                white_material += material_value_map[char]
            else:
                black_material += material_value_map[char]

            col += 1

    legal_moves_layer = 6 if board.turn else 7
    opponent_moves_layer = 7 if board.turn else 6

    for move in board.legal_moves:
        target_col = file_mapping.index(str(move)[2])
        target_row = int(str(move)[3]) - 1
        bitmap[legal_moves_layer, target_row, target_col] = 1

    board.turn = not board.turn

    for move in board.legal_moves:
        target_col = file_mapping.index(str(move)[2])
        target_row = int(str(move)[3]) - 1
        bitmap[opponent_moves_layer, target_row, target_col] = 1

    bitmap[6] = np.flipud(bitmap[6])
    bitmap[7] = np.flipud(bitmap[7])

    return bitmap, white_material, black_material


In [3]:
dataframe = pd.read_csv('chessData.csv')

In [4]:
def create_training_data(sample_size, dataframe):
    """
    Generate training data from a given DataFrame containing FEN strings and evaluations.

    Parameters:
    - sample_size (int): Number of samples to process.
    - dataframe (pd.DataFrame): DataFrame containing 'FEN' and 'Evaluation' columns.

    Returns:
    - x (np.array): Array of shape (samples, 1, 8, 8, 8) representing board states.
    - y (np.array): Array of shape (samples,) containing adjusted evaluations.
    """
    bitmaps = []
    adjusted_evaluations = []

    for index in dataframe.index[:sample_size]:
        row = dataframe.iloc[index]
        
        # Generate compressed bitmap and material difference
        try:
            board_bitmap, white_material, black_material = generate_compressed_bitmaps(row['FEN'])
        except Exception as e:
            print(f"Error processing FEN: {row['FEN']}. Skipping. Error: {e}")
            continue

        material_difference = white_material + black_material

        # Parse evaluation value
        eval_string = row['Evaluation']
        try:
            if '#' in eval_string:  # Checkmate cases
                evaluation = int(eval_string[1:])
            else:
                evaluation = int(eval_string)
        except ValueError:
            print(f"Invalid evaluation: {eval_string}. Skipping.")
            continue

        # Adjust evaluation and add material difference
        adjusted_eval = (evaluation / 100) + material_difference

        # Append data
        bitmaps.append(board_bitmap)
        adjusted_evaluations.append(adjusted_eval)

    # Convert lists to NumPy arrays
    x = np.array(bitmaps, dtype=np.float32)
    x = x.reshape(x.shape[0], 1, 8, 8, 8)  # Reshape for CNN/RNN input
    
    y = np.array(adjusted_evaluations, dtype=np.float32)

    return x, y

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_chess_evaluation_model():
    """
    Builds a Convolutional Neural Network model for chess position evaluation.
    
    Returns:
        model (tf.keras.Model): Compiled Keras model ready for training.
    """
    model = models.Sequential()

    # Input layer with shape (1, 8, 8, 8)
    model.add(layers.Input(shape=(1, 8, 8, 8)))

    # Convolutional layers with ReLU activation and Batch Normalization
    model.add(layers.Conv3D(48, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv3D(48, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv3D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())

    # Flatten layer to convert 2D feature maps to 1D
    model.add(layers.Flatten())

    # Dense layers for final evaluation
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))

    # Output layer with a single neuron for evaluation score
    model.add(layers.Dense(1))

    # Model compilation
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    return model

# Build and summarize the model
chess_model = build_chess_evaluation_model()
chess_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 1, 8, 8, 48)    │        10,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 8, 8, 48)    │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 1, 8, 8, 48)    │        62,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 8, 8, 48)    │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 1, 8, 8, 32)    │        41,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 8, 8, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        65,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181,345 (708.38 KB)

 Trainable params: 181,089 (707.38 KB)

 Non-trainable params: 256 (1.00 KB)

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Đường dẫn để lưu mô hình tốt nhất
checkpoint_path = "best_chess_model.keras"

# Tạo dữ liệu huấn luyện
num_samples = 200000
x_train, y_train = create_training_data(num_samples, dataframe)

# Tạo callback để lưu mô hình tốt nhất dựa trên giá trị mất mát nhỏ nhất trên tập huấn luyện
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='loss', mode='min', verbose=1)

# Huấn luyện mô hình
chess_model.fit(x_train, y_train, epochs=20, batch_size=512, callbacks=[checkpoint_callback])

Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 36.5406 - mse: 36.5406
Epoch 1: loss improved from inf to 32.75726, saving model to best_chess_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 166ms/step - loss: 36.5309 - mse: 36.5309
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - loss: 24.3726 - mse: 24.3726
Epoch 2: loss improved from 32.75726 to 24.09322, saving model to best_chess_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 71s 181ms/step - loss: 24.3719 - mse: 24.3719
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - loss: 18.6466 - mse: 18.6466
Epoch 3: loss improved from 24.09322 to 18.72075, saving model to best_chess_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 73s 188ms/step - loss: 18.6468 - mse: 18.6468
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 14.8442 - mse: 14.8442
Epoch 4: loss improved from 18.72075 to 15.03814, saving model to best_chess_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 75s 191ms/step - loss: 14.8447 - mse: 14.8447
Epoch 5/2